📘 **Note Format Guide**

This format serves as a structured guide for organizing lecture content, personal interpretation, experiments, and study-related questions.

| Type | What It Means | When I Use It |
|------|----------------|----------------|
| 📝 Lecture | Original material from the professor’s notes | When I’m referencing core concepts or provided code |
| 🗣️ In-Class Note | Verbal explanations shared during the lecture | When I want to record something the professor said in class but didn’t include in the official notes |
| ✍️ My Note | My thoughts, interpretations, or additional explanations | When I reflect on or explain something in my own words |
| 🔬 Experiment | Code I tried out or changed to explore further | When I test variations or go beyond the original example |
| ❓ Question | Questions I had while studying | When I want to revisit or research something more deeply |

📝
🗣️
✍️
🔬
❓

# 1. 강의노트 원본 및 영상 링크

[https://guebin.github.io/DL2025/posts/14wk-1.html](https://guebin.github.io/DL2025/posts/14wk-1.html)

# 2. Imports 📝

In [1]:
import gymnasium as gym
#---#
import numpy as np
import collections
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython

# 3. 지난시간 코드 복습 📝

`-` 클래스선언 

- 수정사항: (1)  deque의 maxlen을 500000 으로 조정 (2) print하는 코드를 주석처리 

In [3]:
class GridWorld:
    def __init__(self):
        self.a2d = {
            0: np.array([0,1]), # →
            1: np.array([0,-1]), # ←  
            2: np.array([1,0]),  # ↓
            3: np.array([-1,0])  # ↑
        }
        self.state_space = gym.spaces.MultiDiscrete([4,4])
        self.state = np.array([0,0])
        self.reward = None
        self.terminated = False
    def step(self,action):
        self.state = self.state + self.a2d[action]
        s1,s2 = self.state
        if (s1==3) and (s2==3):
            self.reward = 100 
            self.terminated = True
        elif self.state in self.state_space:
            self.reward = -1 
            self.terminated = False
        else:
            self.reward = -10
            self.terminated = True
        # print(
        #     f"action = {action}\t"
        #     f"state = {self.state - self.a2d[action]} -> {self.state}\t"
        #     f"reward = {self.reward}\t"
        #     f"termiated = {self.terminated}"
        # )            
        return self.state, self.reward, self.terminated
    def reset(self):
        self.state = np.array([0,0])
        self.terminated = False
        return self.state
class RandomAgent:
    def __init__(self):
        self.state = np.array([0,0]) 
        self.action = None 
        self.reward = None 
        self.next_state = None
        self.terminated = None
        #---#
        self.states = collections.deque(maxlen=500000)
        self.actions = collections.deque(maxlen=500000)
        self.rewards = collections.deque(maxlen=500000)
        self.next_states = collections.deque(maxlen=500000)
        self.terminations = collections.deque(maxlen=500000)
        #---#
        self.action_space = gym.spaces.Discrete(4)
        self.n_experience = 0
    def act(self):
        self.action = self.action_space.sample()
    def save_experience(self):
        self.states.append(self.state)
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        self.n_experience = self.n_experience + 1
    def learn(self):
        pass

`-` 메인코드 

- 수정사항: 가독성을 위해 에피소드가 진행되는 for문의 구조를 수정함 (특히 step4)

- 🗣️
    - 성공할 때까지 반복하게 됨 (for => while)
    - score = score + player.reward 가 반복되긴 하지만 terminated 상황을 구분하기 위해 이렇게 작성

:::{.callout-note}
마음에 들지 않지만 꼭 외워야 하는것 

1. `env.step`은 항상 next_state, reward, terminated, truncated, info 를 리턴한다. -- 짐나지엄 라이브러리 규격때문
2. `env.reset`은 환경을 초기화할 뿐만 아니라, state, info를 반환하는 기능도 있다.  -- 짐나지엄 라이브러리 규격때문
3. `player`는 항상 `state`와 `next_state`를 구분해서 저장한다. (다른변수들은 그렇지 않음) 이는 강화학습이 MDP(마코프체인+행동+보상)구조를 따르게 때문에 생기는 고유한 특징이다. -- 이론적이 이유

:::

`-` 환경과 에이전트의 상호작용 이해를 위한 다이어그램: 

- <https://claude.ai/public/artifacts/7fad72b5-0946-47bd-a6cd-b33b21856590> 

# 4. GridWorld 환경의 이해 📝

## A. 데이터 축적

`-` 랜덤에이전트를 이용해 무작위로 100,000 에피소드를 진행해보자. 

In [4]:
player = RandomAgent()
env = GridWorld()
scores = [] 
score = 0 
#
for e in range(1,100000):
    #---에피소드시작---#
    while True:
        # step1 -- 액션선택
        player.act()
        # step2 -- 환경반응 
        player.next_state, player.reward, player.terminated = env.step(player.action)
        # step3 -- 경험기록 & 학습 
        player.save_experience()
        player.learn()
        # step4 --종료 조건 체크 & 후속 처리
        if env.terminated:
            score = score + player.reward
            scores.append(score)
            score = 0 
            player.state = env.reset() 
            break
        else: 
            score = score + player.reward            
            player.state = player.next_state

:::{.callout-important}

***강의노트 수정 2025-06-12***

노규호학생의 도움으로 예전강의의 오류를 발견하여 수정하였습니다. 

```Python
# 수정전
...
        if env.terminated:
            ...
        else: 
            score = score + player.reward
            scores.append(score)            
            player.state = player.next_state
            
# 수정후
        if env.terminated:
            ...
        else: 
            score = score + player.reward
#            scores.append(score)            ### <--- 여기를 주석처리해야함!! 
            player.state = player.next_state
:::

In [5]:
player.n_experience

326581

- 🗣️
    - 한 에피소드 당 평균 3번 정도

## B. 첫번째 `q_table`

`-` 밴딧게임에서는 $q(a)$ 를 정의했었음. 

- $q(0) = 1$
- $q(1) = 10$

`-` 여기에서는 $q(s_1,s_2,a)$를 정의해야함! 

:::{.callout-note}
직관적으로 아래의 그림이 떠오름 
![](https://github.com/guebin/DL2025/blob/main/posts/14wk-1-fig1.png?raw=true)

그림에 대응하는 $q(s_1,s_2,a)$의 값은 아래와 같음

:::{.panel-tabset}

### $a=0$

$a=0 \Leftrightarrow \text{\tt action=right}$

$$ \begin{bmatrix}
q(0,0,0) & q(0,1,0) & q(0,2,0) & q(0,3,0) \\ 
q(1,0,0) & q(1,1,0) & q(1,2,0) & q(1,3,0) \\ 
q(2,0,0) & q(2,1,0) & q(2,2,0) & q(2,3,0) \\ 
q(3,0,0) & q(3,1,0) &q(3,2,0) & q(3,3,0) \\ 
\end{bmatrix} =  \begin{bmatrix}
-1 & -1 & -1 & -10 \\ 
-1 & -1 & -1 & -10 \\ 
-1 & -1 & -1 & -10 \\ 
-1 & -1 & 100 &  \text{-} \\ 
\end{bmatrix}
$$

### $a=1$

$a=1 \Leftrightarrow \text{\tt action=left}$

$$ \begin{bmatrix}
q(0,0,1) & q(0,1,1) & q(0,2,1) & q(0,3,1) \\ 
q(1,0,1) & q(1,1,1) & q(1,2,1) & q(1,3,1) \\ 
q(2,0,1) & q(2,1,1) & q(2,2,1) & q(2,3,1) \\ 
q(3,0,1) & q(3,1,1) &q(3,2,1) & q(3,3,1) \\ 
\end{bmatrix} = \begin{bmatrix}
-10 & -1 & -1 & -1 \\ 
-10& -1 & -1 & -1 \\ 
-10 & -1 & -1 & -1 \\ 
-10 & -1 & -1 &  \text{-} \\ 
\end{bmatrix}
$$

### $a=2$

$a=2 \Leftrightarrow \text{\tt action=down}$

$$  \begin{bmatrix}
q(0,0,2) & q(0,1,2) & q(0,2,2) & q(0,3,2) \\ 
q(1,0,2) & q(1,1,2) & q(1,2,2) & q(1,3,2) \\ 
q(2,0,2) & q(2,1,2) & q(2,2,2) & q(2,3,2) \\ 
q(3,0,2) & q(3,1,2) &q(3,2,2) & q(3,3,2) \\ 
\end{bmatrix} = \begin{bmatrix}
-1 & -1 & -1 & -1 \\ 
-1& -1 & -1 & -1 \\ 
-1 & -1 & -1 & 100\\ 
-10 & -10 & -10 &  \text{-} \\ 
\end{bmatrix}
$$


### $a=3$

$a=3 \Leftrightarrow \text{\tt action=up}$

$$  \begin{bmatrix}
q(0,0,3) & q(0,1,3) & q(0,2,3) & q(0,3,3) \\ 
q(1,0,3) & q(1,1,3) & q(1,2,3) & q(1,3,3) \\ 
q(2,0,3) & q(2,1,3) & q(2,2,3) & q(2,3,3) \\ 
q(3,0,3) & q(3,1,3) &q(3,2,3) & q(3,3,3) \\ 
\end{bmatrix} =\begin{bmatrix}
-10 & -10 & -10 & -10\\ 
-1& -1 & -1 & -1 \\ 
-1 & -1 & -1 & -1 \\ 
-1 & -1 & -1 &  \text{-} \\ 
\end{bmatrix}
$$


:::

:::

`-` 데이터를 바탕으로 $q(s_1,s_2,a)$를 구해보자.

🗣️(

In [6]:
player.states[0], player.actions[0], player.rewards[0]

(array([0, 0]), np.int64(1), -10)

q(0, 0, 1) = -10

✍️ 이 경우에 게임이 끝났음

In [7]:
player.states[1], player.actions[1], player.rewards[1]

(array([0, 0]), np.int64(2), -1)

q(0, 0, 2) = -1

In [8]:
player.states[2], player.actions[2], player.rewards[2]

(array([1, 0]), np.int64(1), -10)

- q_table 만들기

In [11]:
q_table = np.zeros([4,4,4])
q_table

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [12]:
for (s1,s2), a, r in zip(player.states, player.actions, player.rewards):
    q_table[s1,s2,a] = q_table[s1,s2,a] + r # [s1,s2,s3]에서 받은 보상(r)을 계속 더함

In [13]:
q_table[:,:,0]

array([[-29987., -10133.,  -3694., -12710.],
       [-10197.,  -7066.,  -3646., -14840.],
       [ -3761.,  -3648.,  -2312.,  -9980.],
       [ -1361.,  -1446.,  91000.,      0.]])

```
아래와 같은 경우를 기대하였으나 그렇지 않음
-1 -1 -1 -10
-1 -1 -1 -10
-1 -1 -1 -10
-1 -1 100 -
count를 계산하여 평균을 구해야 할 것 같음
```

In [14]:
q_table = np.zeros([4,4,4])
count = np.zeros([4,4,4])

In [15]:
for (s1,s2), a, r in zip(player.states, player.actions, player.rewards):
    q_table[s1,s2,a] = q_table[s1,s2,a] + r
    count[s1,s2,a] = count[s1,s2,a] + 1

In [16]:
q_table[:,:,0]

array([[-29987., -10133.,  -3694., -12710.],
       [-10197.,  -7066.,  -3646., -14840.],
       [ -3761.,  -3648.,  -2312.,  -9980.],
       [ -1361.,  -1446.,  91000.,      0.]])

In [17]:
count[:,:,0]

array([[29987., 10133.,  3694.,  1271.],
       [10197.,  7066.,  3646.,  1484.],
       [ 3761.,  3648.,  2312.,   998.],
       [ 1361.,  1446.,   910.,     0.]])

In [18]:
q_table[...,0] # 동일 코드

array([[-29987., -10133.,  -3694., -12710.],
       [-10197.,  -7066.,  -3646., -14840.],
       [ -3761.,  -3648.,  -2312.,  -9980.],
       [ -1361.,  -1446.,  91000.,      0.]])

In [19]:
count[...,0] # 동일 코드

array([[29987., 10133.,  3694.,  1271.],
       [10197.,  7066.,  3646.,  1484.],
       [ 3761.,  3648.,  2312.,   998.],
       [ 1361.,  1446.,   910.,     0.]])

In [20]:
q_table[...,1]

array([[-301600.,  -10110.,   -3881.,   -1306.],
       [-101040.,   -6958.,   -3589.,   -1430.],
       [ -38240.,   -3584.,   -2295.,    -913.],
       [ -13030.,   -1492.,    -936.,       0.]])

In [21]:
count[...,1]

array([[30160., 10110.,  3881.,  1306.],
       [10104.,  6958.,  3589.,  1430.],
       [ 3824.,  3584.,  2295.,   913.],
       [ 1303.,  1492.,   936.,     0.]])

```
q_table을 count로 나누면 아래와 같은 결과가 나올 것 같음
-10 -1 -1 -1
-10 -1 -1 -1
-10 -1 -1 -1
-10 -1 -1 -
```

In [22]:
q_table / count

/tmp/ipykernel_2313654/1678126644.py:1: RuntimeWarning: invalid value encountered in divide
  q_table / count


array([[[ -1., -10.,  -1., -10.],
        [ -1.,  -1.,  -1., -10.],
        [ -1.,  -1.,  -1., -10.],
        [-10.,  -1.,  -1., -10.]],

       [[ -1., -10.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [-10.,  -1.,  -1.,  -1.]],

       [[ -1., -10.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [-10.,  -1., 100.,  -1.]],

       [[ -1., -10., -10.,  -1.],
        [ -1.,  -1., -10.,  -1.],
        [100.,  -1., -10.,  -1.],
        [ nan,  nan,  nan,  nan]]])

- 없는 값이 존재함

In [24]:
(q_table / count)[...,0]

/tmp/ipykernel_2313654/878353018.py:1: RuntimeWarning: invalid value encountered in divide
  (q_table / count)[...,0]


array([[ -1.,  -1.,  -1., -10.],
       [ -1.,  -1.,  -1., -10.],
       [ -1.,  -1.,  -1., -10.],
       [ -1.,  -1., 100.,  nan]])

- nan 위치의 값은 원래 없으므로 count가 안 됨

In [25]:
count[...,0]

array([[29987., 10133.,  3694.,  1271.],
       [10197.,  7066.,  3646.,  1484.],
       [ 3761.,  3648.,  2312.,   998.],
       [ 1361.,  1446.,   910.,     0.]])

- 0으로 나누는 것이 문제가 되므로 다음과 같이 하면

In [26]:
count[count == 0] = 0.000001
q_table = q_table / count

In [27]:
q_table[...,0]

array([[ -1.,  -1.,  -1., -10.],
       [ -1.,  -1.,  -1., -10.],
       [ -1.,  -1.,  -1., -10.],
       [ -1.,  -1., 100.,   0.]])

)🗣️

In [83]:
player.states[0], player.actions[0], player.rewards[0]

(array([0, 0]), 0, -1)

In [92]:
q_table = np.zeros((4,4,4))
count = np.zeros((4,4,4))

In [93]:
memory =  zip(player.states, player.actions, player.rewards)
for (s1,s2), a, r in memory:
    q_table[s1,s2,a] = q_table[s1,s2,a] + r
    count[s1,s2,a] = count[s1,s2,a] + 1 

In [94]:
count[count==0] = 0.001 

In [95]:
q_table = q_table / count

In [96]:
q_table[...,0], q_table[...,1], q_table[...,2], q_table[...,3]

(array([[ -1.,  -1.,  -1., -10.],
        [ -1.,  -1.,  -1., -10.],
        [ -1.,  -1.,  -1., -10.],
        [ -1.,  -1., 100.,   0.]]),
 array([[-10.,  -1.,  -1.,  -1.],
        [-10.,  -1.,  -1.,  -1.],
        [-10.,  -1.,  -1.,  -1.],
        [-10.,  -1.,  -1.,   0.]]),
 array([[ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1., 100.],
        [-10., -10., -10.,   0.]]),
 array([[-10., -10., -10., -10.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,  -1.],
        [ -1.,  -1.,  -1.,   0.]]))

`-` count를 사용하지 않는 방법은 없을까? -- 테크닉

🗣️(

- `부스팅 알고리즘`을 잘 알면 이해하기 쉬움

```
예시)
어떤 액션을 했을 때 받는 보상 80 / 학습률: 0.1 / 초기값 50
80 - 50 = 30
50 + 3 = 53
80 - 53 = 27
53 + 2.7 = 55.7
...
80 근처로 가게 됨
```

In [28]:
q_table = np.zeros([4,4,4])
for (s1,s2), a, r in zip(player.states, player.actions, player.rewards):
    qhat = q_table[s1,s2,a]
    q = r
    diff = q - qhat
    q_table[s1,s2,a] = q_table[s1,s2,a] + 0.01*diff

In [29]:
q_table

array([[[ -1.        , -10.        ,  -1.        , -10.        ],
        [ -1.        ,  -1.        ,  -1.        , -10.        ],
        [ -1.        ,  -1.        ,  -1.        , -10.        ],
        [ -9.99997166,  -0.99999801,  -0.99999585,  -9.99998066]],

       [[ -1.        , -10.        ,  -1.        ,  -1.        ],
        [ -1.        ,  -1.        ,  -1.        ,  -1.        ],
        [ -1.        ,  -1.        ,  -1.        ,  -1.        ],
        [ -9.99999667,  -0.99999943,  -0.99999928,  -0.9999994 ]],

       [[ -1.        , -10.        ,  -1.        ,  -1.        ],
        [ -1.        ,  -1.        ,  -1.        ,  -1.        ],
        [ -1.        ,  -1.        ,  -1.        ,  -1.        ],
        [ -9.99955952,  -0.9998965 ,  99.99218879,  -0.99983567]],

       [[ -0.99999885,  -9.99997946,  -9.99998584,  -0.99999546],
        [ -0.99999951,  -0.99999969,  -9.9999924 ,  -0.99999986],
        [ 99.98933337,  -0.99991786,  -9.99916195,  -0.99991703],
    

In [30]:
q_table[...,0]

array([[-1.        , -1.        , -1.        , -9.99997166],
       [-1.        , -1.        , -1.        , -9.99999667],
       [-1.        , -1.        , -1.        , -9.99955952],
       [-0.99999885, -0.99999951, 99.98933337,  0.        ]])

In [31]:
q_table[...,0].round(2)

array([[ -1.  ,  -1.  ,  -1.  , -10.  ],
       [ -1.  ,  -1.  ,  -1.  , -10.  ],
       [ -1.  ,  -1.  ,  -1.  , -10.  ],
       [ -1.  ,  -1.  ,  99.99,   0.  ]])

)🗣️

In [109]:
q_table = np.zeros((4,4,4))
memory =  zip(player.states, player.actions, player.rewards)
for (s1,s2), a, r in memory:
    qhat = q_table[s1,s2,a] # 내가 생각했던갓
    q = r # 실제값
    diff = q-qhat # 차이
    q_table[s1,s2,a] = q_table[s1,s2,a]  + 0.01*diff# update

In [112]:
q_table.round(2)

array([[[ -1.  , -10.  ,  -1.  , -10.  ],
        [ -1.  ,  -1.  ,  -1.  , -10.  ],
        [ -1.  ,  -1.  ,  -1.  , -10.  ],
        [-10.  ,  -1.  ,  -1.  , -10.  ]],

       [[ -1.  , -10.  ,  -1.  ,  -1.  ],
        [ -1.  ,  -1.  ,  -1.  ,  -1.  ],
        [ -1.  ,  -1.  ,  -1.  ,  -1.  ],
        [-10.  ,  -1.  ,  -1.  ,  -1.  ]],

       [[ -1.  , -10.  ,  -1.  ,  -1.  ],
        [ -1.  ,  -1.  ,  -1.  ,  -1.  ],
        [ -1.  ,  -1.  ,  -1.  ,  -1.  ],
        [-10.  ,  -1.  ,  99.99,  -1.  ]],

       [[ -1.  , -10.  , -10.  ,  -1.  ],
        [ -1.  ,  -1.  , -10.  ,  -1.  ],
        [ 99.99,  -1.  , -10.  ,  -1.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ]]])

## C. 첫번째 `q_table`보다 나은 것?

`-` 첫번째 `q_table`을 알고있다고 가정하자. 

![](https://github.com/guebin/DL2025/blob/main/posts/14wk-1-fig1.png?raw=true)

`-` 정책시각화 (합리적인 행동) 

![](https://github.com/guebin/DL2025/blob/main/posts/14wk-1-fig2.png?raw=true)

`-` 이게 최선의 정책일까?

- 🗣️
    - 큰 숫자 따라가기?
    - (2,2)에서 오른쪽과 아래는 -1 이 아니라 99로 수정?
        - 오른쪽이나 아래로 가면 100이 보장이 되어 있음
    - (1,2)에서 왼쪽과 아래쪽은 reward가 똑같은 것이 맞는가?
        - 아래로 가면 99가 보장이 되어 있으므로 98로 수정?
    - 이런 식으로 100점을 기준으로 점차 재조정이 되어야 할 것 같음